<a href="https://colab.research.google.com/github/inesbsilveira/hummingbirds/blob/main/ARR/ARR_eligibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.Imports

In [ ]:
!pip install geemap
!pip install geojson

In [2]:
import os
import csv
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.validation import make_valid
import geojson
import zipfile
import ee
import geemap

### 2.Connect to GEE

In [3]:
my_project = 'ee-ineshummingbirds'
ee.Authenticate()
ee.Initialize(project= my_project)

### 3.Input file



In [4]:
# if the input is a shapefile
input_shp = "original_calao_shp.shp"
gdf = gpd.read_file(input_shp).to_crs('EPSG:4326')
# if the input is a geojson
#input_geojson = "NSVK_cleaned_17_35.geojson"
#gdf = gpd.read_file(input_geojson).to_crs('EPSG:4326')
File = geemap.geopandas_to_ee(gdf)

### 4.Variables

In [5]:
# Project
country = 'Ivory Coast'
project_area_name = 'Calao' #region/country/project name

#dates for landsat for 2014 and 2024
year_0 = 2014    # start year
year_10 = 2024  # end year
start_date = '-01-01'  # -mm-dd format // beginning of dry season
end_date = '-03-30'   # -mm-dd format // end of dry seaosn

#dates for landsat for 2020
year_0_2020 = 2019
year_1_2020 = 2020
start_date_2020 = '-12-01' # -mm-dd format
end_date_2020 = '-02-01' # -mm-dd format

#slope
slope_percentage = 30 # in percentage

#minimum forest size for eligibility (1ha=11pixels)
min_forest_pixels_list = [11, 55, 110] #1ha, 5ha, and 10ha

In [6]:
#forest variables // The project's country has to be in the excel file 'Countries_Forest_Definition.xlsx'. Add this data if it is not already in the doc
df_forest_definition = pd.read_excel('Countries_Forest_Definition.xlsx') #read the excel file
country_data = df_forest_definition[df_forest_definition['Country'] == country] #filter the country
cover_threshold = country_data['Tree_crown_cover_%'].values[0] #get cover percentage from doc
cover_threshold = int(cover_threshold)
height_threshold = country_data['Tree_height_m'].values[0] #get tree height from doc
height_threshold = int(height_threshold)
forest_size_ha = country_data['Area_ha'].values[0] #get minimum forest size from doc

#forest size (1ha=11pixels)
forest_size_pixels = forest_size_ha * 11

### 5.Functions and Legends

##### Functions

In [7]:
#Define a function to reclassify the 'tree' class of ESA
def reclassify(image):
    return image.where(image.eq(10), result)

#### Pre-process Landsat
def apply_scale_factors (image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Mask the clouds (if the image has clouds it will delete the pixel)
def maskSrClouds(image):
    # Select the QA_PIXEL band and create a mask to exclude cloudy pixels
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    # Select the QA_RADSAT band and create a mask to exclude saturated pixels
    saturation_mask = image.select('QA_RADSAT').eq(0)
    # Apply both masks to the image
    masked_image = image.updateMask(qa_mask).updateMask(saturation_mask)
    return masked_image

## Fill the cloud missing values
def fillGap(image):
  return image.focalMedian(1.5, 'square', 'pixels', 2).blend(image)

## Rename function to consistenty have the same band names among Landsat-7, Landsat-8 and landsat-9
def rename(image):
  return image.select(
      ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],
      ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

def renamel9(image):
  return image.select(
      ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],
      ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

visParams={
   'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
   'min': 0,
   'max': 0.2
}

# Function to create the Landsat image collection
def create_landsat_collection(collection, start_year, end_year, start_date, end_date, region_of_interest, apply_scale_factors, maskSrClouds):
    """
    Creates a Landsat image collection for a specific year, date range, and region of interest.

    Args:
        year (int): The year for filtering.
        start_date (str): The start date for the image collection in "yyyy-mm-dd" format.
        end_date (str): The end date for the image collection in "yyyy-mm-dd" format.
        region_of_interest (ee.Geometry): The region to filter the images by.
        apply_scale_factors (function): A function to apply scale factors to the images.
        mask_sr_clouds (function): A function to mask out cloud-covered pixels.

    Returns:
        ee.ImageCollection: The filtered Landsat image collection.
    """
    collection = (
        ee.ImageCollection(collection)
        .filterBounds(region_of_interest)
        .filterDate(str(start_year) + start_date, str(end_year) + end_date)
        .filter(ee.Filter.lt('CLOUD_COVER', 30))
        .map(apply_scale_factors)
        .map(maskSrClouds)
    )

    return collection


def create_landsat_collection_with_clouds(collection, start_year, end_year, start_date, end_date, region_of_interest, apply_scale_factors, maskSrClouds, renamel9):
  collection_l7 = (
    ee.ImageCollection(collection)
    .filterBounds(region_of_interest)
    .filterDate(str(start_year) + start_date, str(end_year)+ end_date)
    .filter(ee.Filter.lt('CLOUD_COVER', 50))
    .map(apply_scale_factors)
    .map(maskSrClouds)
    .map(renamel9))
  return collection_l7

def create_composite(collection_l7, collection, fillGap, File):
    """
    General function to process Landsat image collections, merge, apply gap filling, and create a composite.

    Parameters:
    - collection_l7: Landsat image collection with cloud missing values.
    - collection: The primary Landsat image collection to use for band selection.
    - fillGap: Function to apply gap filling.
    - File: Geometry or boundary to clip the output image.

    Returns:
    - A median composite image clipped to the specified boundary.
    """
    # Merge the collections based on the selected bands
    landsat78 = collection_l7.merge(collection.select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6','SR_B7']))

    # Apply gap filling and merge the collections again
    composite78 = landsat78.map(fillGap).merge(collection.select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6','SR_B7']))

    # Compute the median and clip the result
    landsat78composite = composite78.median().clip(File)

    return landsat78composite


def get_shapefile_centroid(gdf):
    """Ensure CRS is geographic and return the centroid coordinates."""
    if gdf.crs is None or gdf.crs.is_projected:
        gdf = gdf.to_crs(epsg=4326)  # Convert to WGS84 (lat/lon)

    centroid = gdf.unary_union.centroid
    return centroid.y, centroid.x  # (latitude, longitude)

def get_best_crs(latitude, longitude):
    """ Returns the best UTM zone EPSG code based on latitude """
    utm_zone = int((180 + longitude) / 6) + 1
    return f"EPSG:{32600 + utm_zone if latitude >= 0 else 32700 + utm_zone}"


def calculateTotalPixelArea(image, geometry):
    """
    Calculates the total area in hectares of an image within a given geometry.

    Parameters:
    - image: ee.Image - The input classified image.
    - geometry: ee.Geometry - The region of interest.

    Returns:
    - Dictionary with area values or None in case of an error.
    """

    # Ensure the image is in a projected CRS (Web Mercator or UTM)
    image = image.reproject(best_epsg, None, 30)

    # Compute pixel area in hectares
    total_area = ee.Image.pixelArea().addBands(image).divide(10_000).reduceRegion(
        reducer=ee.Reducer.sum().group(1),  # Sum areas by class
        geometry=geometry,
        scale=30,
        bestEffort=True,
        tileScale=16  # Reduce memory usage
    )

    # Retrieve results
    try:
        result = total_area.getInfo()
        if not result:
            print("No area data found.")
            return None

        # Convert to a DataFrame
        df = pd.DataFrame.from_dict(result, orient='columns')
        print(df)
        return result

    except Exception as e:
        print(f"Error encountered: {e}")
        return None


##### Legends

In [8]:
esa_legend_dict = {
    'Forest': '006400',
    'Shrubland': 'ffbb22',
    'Grassland': 'ffff4c',
    'Cropland': 'f096ff',
    'Built-up': 'fa0000',
    'Bare/sparse vegetation': 'b4b4b4',
    'Snow and ice': 'f0f0f0',
    'Permanent water bodies': '0064c8',
    'Herbaceous wetland': '0096a0',
    'Mangroves': '00cf75',
    'Moss and lichen': 'fae6a0'
}

legend_dict = {
    'Forest': '006400',
    'Non-forest': 'ffff4c',
    'Built-up': 'fa0000',
    'Permanent water bodies': '0064c8',
    'Other land': '0096a0',
}

### 6.Main

##### Get ESA WORLD COVER classification

In [9]:
gfc = ee.Image('UMD/hansen/global_forest_change_2023_v1_11').clip(File);
canopyCover = gfc.select(['treecover2000']).clip(File).gte(cover_threshold)

# Tree height
tree_height = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1').clip(File.geometry()).gte(height_threshold)

# Overlay tree cover and tree height to have a layer presenting the threshold
forest_mask = tree_height.multiply(canopyCover)  # Overlay the two layers

#Load the ESA WorldCover and clip the area of interest ##IF NOT HAVING FOREST DEFINITION BY THE COUNTRY and small project area <100.000 ha
esa = ee.ImageCollection('ESA/WorldCover/v200').first().clip(File.geometry())
#Create a binary image from ESA just for tree (10)
esa_10 = esa.eq(10)
#Overlay ESA forest image and canopy cover
result = esa_10.multiply(forest_mask)

In [10]:
reclassify_map = reclassify(esa)
# Reclassify the reference land-use map
forest = reclassify_map.eq(10).selfMask().multiply(1)
non_forest = reclassify_map.eq(0).Or(reclassify_map.eq(20)).Or(reclassify_map.eq(30)).Or(reclassify_map.eq(40)).selfMask().multiply(2)
built_up = reclassify_map.eq(50).selfMask().multiply(3)
water = reclassify_map.eq(80).selfMask().multiply(4)
other_land = reclassify_map.eq(60).Or(reclassify_map.eq(70)).Or(reclassify_map.eq(90)).Or(reclassify_map.eq(95)).Or(reclassify_map.eq(100)).selfMask().multiply(5)

new_esa = forest.blend(non_forest).blend(built_up).blend(water).blend(other_land)

In [ ]:
###If runing only ESA the code is:
Map=geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(File, 10)
#Map.addLayer(new_esa, {'min':1, 'max': 5,'palette': ['006400', 'ffff4c', 'fa0000', '0064c8', '0096a0']}, 'ESA')
#Map.add_legend(title='Land Cover', legend_dict = legend_dict)
Map.addLayer(esa, {'bands':['Map']}, 'ESA')
Map

In [ ]:
# Save tif to Google Drive
geemap.ee_export_image_to_drive(
    esa, description=f'{project_area_name}_ESA_map', region=File.geometry(), scale=30
)

##### Train the model and get estimated ESA World Cover for 2014 and 2024

In [14]:
#get sample points for training and validation
#Change according to the sampling method
points = esa.sample(
    **{
        "region": File.geometry(),
        "scale": 30,
        "numPixels": 10000,
        "seed": 0,
        "geometries": True,
    })

In [15]:
#Landsat collection for year 0
collection_y0 = create_landsat_collection('LANDSAT/LC08/C02/T1_L2',year_0, year_0, start_date, end_date, File, apply_scale_factors, maskSrClouds)
collection_l7_y0 = create_landsat_collection_with_clouds('LANDSAT/LE07/C02/T1_L2', year_0, year_0, start_date, end_date, File, apply_scale_factors, maskSrClouds, rename)
landsat78composite_y0 = create_composite(collection_l7_y0, collection_y0, fillGap, File)

print('Number of Landsat scenes for year 0:', collection_y0.size().getInfo())
print('Number of Landsat scenes for year 0:', collection_l7_y0.size().getInfo())

Number of Landsat scenes for year 0: 4
Number of Landsat scenes for year 0: 11


In [17]:
#Landsat collection for year 10
collection_y10 = create_landsat_collection('LANDSAT/LC08/C02/T1_L2', year_10, year_10, start_date, end_date, File, apply_scale_factors, maskSrClouds)
collection_l7_y10 = create_landsat_collection_with_clouds('LANDSAT/LC09/C02/T1_L2', year_10, year_10, start_date, end_date, File, apply_scale_factors, maskSrClouds, renamel9)
landsat78composite_y10 = create_composite(collection_l7_y10, collection_y10, fillGap, File)

print('Number of Landsat scenes for year 10:', collection_y10.size().getInfo())
print('Number of Landsat scenes for year 10:', collection_l7_y10.size().getInfo())

Number of Landsat scenes for year 10: 9
Number of Landsat scenes for year 10: 8


In [18]:
#Landsat collection for 2020
collection_2020 = create_landsat_collection('LANDSAT/LC08/C02/T1_L2', year_0_2020, year_1_2020, start_date_2020, end_date_2020, File, apply_scale_factors, maskSrClouds)
collection_l7_2020 = create_landsat_collection_with_clouds('LANDSAT/LE07/C02/T1_L2', year_0_2020, year_1_2020, start_date_2020, end_date_2020, File, apply_scale_factors, maskSrClouds, rename)
landsat78composite_2020 = create_composite(collection_l7_2020, collection_2020, fillGap, File)

print('Number of Landsat scenes for year 2020:', collection_2020.size().getInfo())
#print('Number of Landsat scenes for year 2020:', collection_l7_2020.size().getInfo())

Number of Landsat scenes for year 2020: 12


In [ ]:
#check the cloud cover in the retrieved scenes
Map=geemap.Map()
Map.add_basemap('HYBRID', False)
Map.centerObject(File, 10)
Map.addLayer(landsat78composite_y0, visParams, 'Composite year 0')
Map.addLayer(landsat78composite_y10, visParams, 'Composite year 10')
Map.addLayer(landsat78composite_2020, visParams, 'Composite year 2020')
Map

In [20]:
bands = ['SR_B2','SR_B3','SR_B4','SR_B5', 'SR_B6', 'SR_B7']
label = 'Map'       # Label and bands are staying the same despite the year

training_2020 = landsat78composite_2020.select(bands).sampleRegions (**{
    'collection' : points,
    'properties' : [label],
    'scale'      : 30  # 30 m resolution based on the Landsat 8 resolution
})

# Add a column for the accuracy assessment
training_2020 = training_2020.randomColumn()

training_2020_new = training_2020.filter(ee.Filter.lt('random', 0.7))
validation_2020_new = training_2020.filter(ee.Filter.gte('random', 0.7))

# Using the Classifier.smilecart machine learning to predict and classify the land cover
trained_2020 = ee.Classifier.smileRandomForest(10).train(training_2020_new, label, bands)# Train the classifier using the trianing data generated

# Reclassifying the image classes + values
result_y10 = landsat78composite_y10.select(bands).classify(trained_2020) # classify the image/raster
result_y0 = landsat78composite_y0.select(bands).classify(trained_2020)


In [21]:
##FOR ESA
# Recalculate the forest class according to the new stand-size
forest_y10 = result_y10.eq(10).selfMask()
forest_y0 = result_y0.eq(10).selfMask()

contArea_y10 = forest_y10.connectedPixelCount()
area_y10 = contArea_y10.gte(forest_size_pixels).selfMask()

contArea_y0 = forest_y0.connectedPixelCount()
area_y0 = contArea_y0.gte(forest_size_pixels).selfMask()

reclassify_y10 = result_y10.where(result_y10.eq(10), area_y10)
reclassify_y0 = result_y0.where(result_y0.eq(10), area_y0)


In [ ]:
left_layer = geemap.ee_tile_layer(forest_y10, {'bands': ['classification']}, 'Landcover year 10')
right_layer = geemap.ee_tile_layer(forest_y0, {'bands': ['classification']}, 'Landcover year 0')
Map1 = geemap.Map()
Map1.centerObject(File,12)
Map1.split_map(left_layer, right_layer)
Map1.add_legend(title='Land Cover', legend_dict = esa_legend_dict)
Map1

In [22]:
###IF USING ESA
class_values = esa.get('Map_class_values')#.getInfo()
class_palette = esa.get('Map_class_palette')#.getInfo()
class_names = esa.get('Map_class_names')
# Reclassifying the class using the original class names and class palette
landcover_y10 = reclassify_y10.set ('classification_class_values', class_values)
landcover_y10 = landcover_y10.set('classification_class_palette', class_palette)
landcover_y10 = landcover_y10.set('classification_class_names', class_names)
landcover_y0 = reclassify_y0.set ('classification_class_values', class_values)
landcover_y0 = landcover_y0.set('classification_class_palette', class_palette)
landcover_y0 = landcover_y0.set('classification_class_names', class_names)

In [ ]:
left_layer = geemap.ee_tile_layer(landcover_y10, {'bands': ['classification']}, 'Landcover year 10')
right_layer = geemap.ee_tile_layer(landcover_y0, {'bands': ['classification']}, 'Landcover year 0')
Map1 = geemap.Map()
Map1.centerObject(File,12)
Map1.split_map(left_layer, right_layer)
Map1.add_legend(title='Land Cover', legend_dict = esa_legend_dict)
Map1

In [ ]:
# Save tif to Google Drive
#2014
geemap.ee_export_image_to_drive(
    landcover_y0, description=f'{project_area_name}_ESA_map_2014', region=File.geometry(), scale=30
)

In [ ]:
#2024
geemap.ee_export_image_to_drive(
    landcover_y10, description=f'{project_area_name}_ESA_map_2024', region=File.geometry(), scale=30
)

##### Accuracy of the model

In [23]:
#Overall accuracy
#Training dataset
training_accuracy = trained_2020.confusionMatrix()
overall_accuracy = training_accuracy.accuracy()
print(overall_accuracy.getInfo())

# Accuracy on Validation dataset
validation = validation_2020_new.classify(trained_2020)
validation.first().getInfo()

validation_accuracy = validation.errorMatrix('Map', 'classification')

#print('The accuracy is:')
print(validation_accuracy.accuracy().getInfo())

0.9465724381625442
0.6584615384615384


##### Calculate the slope and its eligibility

In [24]:
##FOR ESA ONLY
non_forest_y0 = reclassify_y0.eq(20).Or(reclassify_y0.eq(30)).Or(reclassify_y0.eq(40)).Or(reclassify_y0.eq(60))
non_forest_y10 = reclassify_y10.eq(20).Or(reclassify_y10.eq(30)).Or(reclassify_y10.eq(40)).Or(reclassify_y10.eq(60))
forest_y0 = reclassify_y0.eq(10).selfMask()
forest_y10 = reclassify_y10.eq(10).selfMask()

In [25]:
## slope

dataset = ee.Image('USGS/SRTMGL1_003').select('elevation')
slope = ee.Terrain.slope(dataset)                                                    # Getting the slope
#possible to change the slope in here (if not consider just change to 0)
slope_30 = slope.updateMask(slope.gt(30)).updateMask(slope.lt(100)).gt(30)
#### Getting the difference between the land cover in 2023 and 2013 by overlaying those 2 rasters on each other - Multiplication "Py language" ####

# Overlaying to see the differences between 2023 and 2013
#overlayed_10year = reclassify_y0.multiply(reclassify_y10)

# Overlaying to see the differences between 2023 and 2013 ONLY FOR ESA
overlayed_10year = non_forest_y0.multiply(non_forest_y10)

# Overlaying with slope
Slope_Mul_Overlayed = slope_30.multiply(overlayed_10year.eq(10))

Map2 = geemap.Map()
Map2.centerObject(File)
Map2.addLayer (File, {}, 'File')
Map2.addLayer(overlayed_10year.selfMask(),{'min': 0, 'max':3, 'palette': ['green','green']}, "Eligible Areas - Green")
Map2.addLayer(Slope_Mul_Overlayed.selfMask(),{'min': 0, 'max':3, 'palette': ['red','red']}, "non-eligible - slope > 30%")
Map2.centerObject(File, 11)
Map2

Map(center=[7.843112381185676, -4.717413380772332], controls=(WidgetControl(options=['position', 'transparent_…

### 7.Results

##### Areas calculation

In [ ]:
latitude, longitude = get_shapefile_centroid(gdf)
#print(f"Central Point: ({latitude}, {longitude})")
best_epsg = get_best_crs(latitude, longitude)  # Replace with actual latitude
print(best_epsg)

In [ ]:
gdf_crs = gdf.to_crs(best_epsg)
total_area_ha = (gdf_crs['geometry'].area/10000).sum()
print(f"Total area in hectares: {total_area_ha}")

In [ ]:
#CALCULATE THE FOREST AREA FOR YEAR 0 AND YEAR 10
print('Forest year 0:')
forest_year0 = calculateTotalPixelArea(forest_y0, File)

print('Forest year 10:')
forest_year10 = calculateTotalPixelArea(forest_y10, File)

##### Eligibility

In [ ]:
#Calculate total eligible area
print('Non-eligible and eligible area are:')
eligible_area = calculateTotalPixelArea(overlayed_10year, File)

#Non-eligible slope
#print('Slopy area:')
#non_eligible_slope = calculateTotalPixelArea(Slope_Mul_Overlayed.selfMask(), File)

In [ ]:
# If the shapefile is divided into multiple polygons of interest (departments, geopolitical units, etc), use the code below

# Create a list to hold the features
features = []

# Get the size of the File FeatureCollection
file_size = File.size().getInfo()

# Retrieve the features using a loop
for i in range(file_size):
    feature = ee.Feature(File.toList(file_size).get(i))
    features.append(feature)

# Calculate the total pixel area for each feature
print('Eligible area:')
areas = [calculateTotalPixelArea(overlayed_10year, feature.geometry()) for feature in features]

print('Slopy area:')
areas_slope = [calculateTotalPixelArea(Slope_Mul_Overlayed.selfMask(), feature.geometry()) for feature in features]

In [ ]:
# Define different min_forest_pixels values
# Store results in a list
results = []

for min_forest_pixels in min_forest_pixels_list:
    contArea = overlayed_10year.eq(1).selfMask().connectedPixelCount()
    area = contArea.gte(min_forest_pixels).selfMask()
    areas_slope = calculateTotalPixelArea(area.selfMask(), File)

    # Append results to the list
    results.append({"min_forest_pixels": min_forest_pixels, "Total Area (ha)": areas_slope})

# Convert to DataFrame for a structured table
df = pd.DataFrame(results)

# Print the table
print(df)

In [ ]:
# Save tif to Google Drive
geemap.ee_export_image_to_drive(
    overlayed_10year, description=f'{project_area_name}_eligible_area', region=File.geometry(), scale=30
)